In [28]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

#import beautiful to scrape data
from bs4 import BeautifulSoup

#open url
from urllib.request import urlopen

print('Libraries imported.')

Libraries imported.


In [29]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

In [30]:
tables = soup.find_all('table')

In [53]:
postcode = []
bor = []
neig = []

for table in tables:
    rows = table.find_all('tr')   
    for row in rows:
        cells = row.find_all('td')    
       
        if len(cells) == 3:
            code = cells[0]
            postcode.append(code.text.rstrip('\n'))

            borough = cells[1]
            bor.append(borough.text.rstrip('\n'))

            neighborhood = cells[2]
            neig.append(neighborhood.text.rstrip('\n'))

In [66]:
df= pd.DataFrame(postcode,columns=['Postal Code'])
df['Borough'] = bor
df['Neighborhood'] = neig
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [98]:
df_new = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df_new.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Part 2 - Get the latitude and the longitude coordinates of each neighborhood

In [78]:
data = pd.read_csv('http://cocl.us/Geospatial_data')
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [92]:
from geopy import geocoders
from geopy.geocoders import Nominatim

In [ ]:
df_new['Postal Code']

In [93]:
g = geocoder.arcgis('M5G')
print(g.latlng)
print(type(g.latlng))

[43.65609000000006, -79.38492999999994]
<class 'list'>


In [96]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format('M5G'))
    lat_lng_coords = g.latlng
    
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
                              
print(latitude)
print(longitude)

43.65609000000006
-79.38492999999994


In [106]:
data.reset_index(drop=True, inplace=True)
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [109]:
lat_lng_coords = []

for find in df_new["Postal Code"]:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(find))
    lat_lng_coords.append(g.latlng)

print(lat_lng_coords)

[[43.75188000000003, -79.33035999999998], [43.73042000000004, -79.31281999999999], [43.655140000000074, -79.36264999999997], [43.72321000000005, -79.45140999999995], [43.66449000000006, -79.39301999999998], [43.66277000000008, -79.52830999999998], [43.81153000000006, -79.19551999999999], [43.74929000000003, -79.36168999999995], [43.707940000000065, -79.31159999999994], [43.65736000000004, -79.37817999999999], [43.70799000000005, -79.44837999999999], [43.65279000000004, -79.55405999999994], [43.78564000000006, -79.15870999999999], [43.72184000000004, -79.34339999999997], [43.68970000000007, -79.30679999999995], [43.65143000000006, -79.37556999999998], [43.69211000000007, -79.43035999999995], [43.648900000000026, -79.57824999999997], [43.765750000000025, -79.17519999999996], [43.67703000000006, -79.29541999999998], [43.64531000000005, -79.37367999999998], [43.68784000000005, -79.45045999999996], [43.768200000000036, -79.21760999999998], [43.70909000000006, -79.36409999999995], [43.656090

In [112]:
location = pd.DataFrame(lat_lng_coords)
location.columns = ['Latitude', 'Longitude'] 
location.head()

,Latitude,Longitude
0,43.75188,-79.33036
1,43.73042,-79.31282
2,43.65514,-79.36265
3,43.72321,-79.45141
4,43.66449,-79.39302


In [115]:
df_final=pd.concat([df_new, location], axis=1)
df_final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75188,-79.33036
1,M4A,North York,Victoria Village,43.73042,-79.31282
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72321,-79.45141
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302


# Part 3

In [118]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


In [128]:
df_map=df_final[df_final.Borough.str.endswith('Toronto')]
df_map.reset_index(inplace=True, drop=True)
df_map1 = pd.DataFrame(df_map)
df_map1.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818
3,M5C,Downtown Toronto,St. James Town,43.65143,-79.37557
4,M4E,East Toronto,The Beaches,43.67703,-79.29542


In [131]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_map1['Latitude'], df_map1['Longitude'], df_map1['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto) 

map_Toronto